# 0.初めに
このノートブックでは学習済みモデルの精度を実際に確認できるようにしてあります。
もし学習済みモデルをダウンロードしていない場合はhttps://drive.google.com/file/d/1-4dLB4T9h0XP-rpvYnbXvxBBnXW1vegi/view?usp=sharing
からダウンロードして、google driveのマイドライブ直下に保存してください。

このノートブックは1.モデルを試す　からが本質になっているため、そこまではとりあえず実行してください。

In [ ]:
#このセルはとりあえず実行してください
!git clone https://github.com/huggingface/transformers
!git checkout efdb46b6e2a53f9126d447260f916cac33de58c3
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir apex/
!pip install mecab-python3==0.996.6rc2
!pip install transformers==2.8.0
!git clone https://github.com/kuma807/bert_qa.git

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#もしgoogle driveのマイドライブ直下以外に保存した場合model_dirを変更してください
model_dir = "gdrive/My Drive/model/pytorch_model.bin"

In [5]:
#モデルと予想用関数の設定
from transformers import BertTokenizer, BertForQuestionAnswering, AutoTokenizer, AutoConfig
import torch
config = AutoConfig.from_pretrained("bert_qa/config.json")
tokenizer_config = AutoConfig.from_pretrained("bert_qa/tokenizer_config.json")
model = BertForQuestionAnswering.from_pretrained("bandainamco-mirai/distilbert-base-japanese", config=config)
model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", config=tokenizer_config)
def predict(quesion, text):
  input_ids = tokenizer.encode(quesion, text)
  token_type_ids = [0 if i <= input_ids.index(3) else 1 for i in range(len(input_ids))]
  start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
  score = torch.max(start_scores).item() + torch.max(end_scores).item()
  all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  prediction = ''.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
  prediction = prediction.replace("##", "")
  return prediction, score

# 1.モデルを試す
textにモデルに与える問題文、questionに質問を入れることでモデルの予想を見ることができます。
下に2つほど問題文と質問の例を示していますが、基本的にどんな質問でも行けると思うので自分で質問を作ったりして試してみてください。
モデルの予想に[CLS]と出た場合それはモデルが問題文に答えがないと判断していると思ってください。

In [6]:
#例1
text = """
水物である、水が1Lと、牛乳が0.5Lと、オレンジジュースが0.2L、キャベツが0.7個あります。
"""
quesion = ""

prediction, score = predict(quesion, text)
print("モデルの予想結果:", prediction)

モデルの予想結果: [CLS]


In [ ]:
#例2
text = """
紀元前10世紀ごろに成立したと見られるヤハウェ信仰者による出典によると、ヤハウェは地が乾きなにも生えていないころに最初にアダムを創造したとされる。
ヤハウェは地面の土（アダマ）を使ってアダムの形を作り、鼻の穴からルーアハを吹き込んだ。
"""

quesion = "りんごは何色ですか？"
prediction, score = predict(quesion, text)
print("モデルの予想結果:", prediction)

モデルの予想結果: [CLS]


例2では問題文中に答えがなくモデルが[CLS]を出力する